# Alternating Minimization on Real Ranking Data

In [207]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from scipy.linalg import orthogonal_procrustes
from sklearn.metrics import mean_squared_error
from helpers.helpers_pmf import *

In [208]:
# Set the mean and standard deviations for the distributions
mu = 0.0
sigma_u = 1.0
sigma_v = 1.0
sigma = 0.2

# Create an empty dictionary to store the parameters
parameters = {}

In [210]:
group_0_df = pd.read_csv('data/group_0.csv')
group_1_df = pd.read_csv('data/group_1.csv')
group_2_df = pd.read_csv('data/group_2.csv')
group_3_df = pd.read_csv('data/group_3.csv')
group_4_df = pd.read_csv('data/group_4.csv')
ratings_df = pd.read_csv('data/ratings_per_user.csv')
ratings_df.head()

,user_hash,0071.jpg,0351.jpg,0428.jpg,0546.jpg,0001.jpg,0290.jpg,0527.jpg,0054.jpg,0652.jpg,...,0636.jpg,0409.jpg,0315.jpg,0741.jpg,0356.jpg,0650.jpg,0535.jpg,0382.jpg,0611.jpg,0128.jpg
0,2u7T5g3WNhXLNt8eXLm7iYrvR872,4.0,5.0,5.0,3.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3mHQgTNdhoXXOWEsJmHvbBuOxSb2,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4jQ4QWrm9SOsFMLIAajSZ5Usva82,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5HBSsBrc7LSwIQyRgTqY7Zf641f2,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,69AHmihCRKbJbXXM9GQ4FT7NSU32,5.0,4.0,2.0,5.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [211]:
def preprocess_data(ratings_df):
    # Set the user_hash column as the index of the DataFrame
    ratings_matrix_df = ratings_df.set_index('user_hash')

    # Convert the DataFrame into a numpy array
    ratings_matrix = ratings_matrix_df.values

    # Get the number of users and items
    n_users = ratings_matrix.shape[0]
    n_items = ratings_matrix.shape[1]

    return ratings_matrix, n_users, n_items


We will now initialise our parameters. First the $V$ matrix can be initialised randomly using the following distribution:

\begin{equation}
\large
V \sim \mathcal N\left(0, \frac {1} {\lambda_V}\right)
\end{equation}

Let's remember that:

\begin{equation}
\large
U \in \mathbb R^{D\times N}, \qquad V \in \mathbb R^{D\times M}
\end{equation}

Where $N$ is __n_users__, $M$ is __n_movies__ and $D$ is __n_dims__.

In [192]:
def init_params_real_data(sigma_U, sigma_V, mu, sigma, d_dims, n_users, n_movies, ratings_matrix):    
    # Initialize U and V with random values
    U_init = np.random.normal(mu, sigma_U, (d_dims, n_users))
    V_init = np.random.normal(mu, sigma_V, (d_dims, n_movies))    
    
    # Set the ratings matrix, U, V, and lambda values in the parameters dictionary
    parameters['R'] = ratings_matrix
    parameters['U_result'] = U_init
    parameters['V_result'] = V_init
    parameters['lambda_U'] = sigma**2/sigma_U**2
    parameters['lambda_V'] = sigma**2/sigma_V**2


Create __ratings__ dataframe and append rating matrix

In [193]:
def create_real_data_df(n_users, n_movies):   
    
    df = pd.DataFrame(columns=['userID', 'movieID'])
    df['userID'] = np.repeat(np.arange(1, n_users+1, 1), n_movies)
    df['movieID'] = np.tile(np.arange(1, n_movies+1, 1), n_users)     
    R = parameters['R']
    df['rating'] = R.reshape(n_users*n_movies)

    return df 

__Split__ the dataset into train and test sets

In [194]:
def split_train_test(df, train_size=0.75):
    df_copy = df.copy()
    
    # Sample a fraction of the DataFrame for the training set
    train_set = df_copy.sample(frac=train_size, random_state=0)
    
    # Remove the samples in the training set from the copy to create the testing set
    test_set = df_copy.drop(train_set.index)
    
    return train_set, test_set


Let's now implement the function that updates U and V. The elements of both matrices can be updated using the following expressions:

\begin{equation}
\large
U_i=\left[\left(V_jV_j^T\right)_{j\in\Omega_{U_i}}+\lambda_UI\right]^{-1}\left(R_{ij}V_j^T\right)_{j\in\Omega_{U_i}}
\end{equation}

\begin{equation}
\large
V_j=\left[\left(U_iU_i^T\right)_{i\in\Omega_{V_j}}+\lambda_VI\right]^{-1}\left(R_{ij}U_i^T\right)_{i\in\Omega_{V_j}}
\end{equation}

In [195]:
def update_parameters(n_users, n_movies, d_dims):
    U = parameters['U_result']
    V = parameters['V_result']
    R = parameters['R']
    lambda_U = parameters['lambda_U']
    lambda_V = parameters['lambda_V']

    # Update U for each user
    for i in range(n_users):
        # Select the items for which the user has provided ratings
        V_j = V[:, R[i, :] > 0]
        
        # Compute the updated U for the user
        U[:, i] = np.dot(np.linalg.inv(np.dot(V_j, V_j.T) + lambda_U * np.identity(d_dims)),
                         np.dot(R[i, R[i, :] > 0], V_j.T))
        
    # Update V for each movie
    for j in range(n_movies):
        # Select the users who have rated the movie
        U_i = U[:, R[:, j] > 0]
        
        # Compute the updated V for the movie
        V[:, j] = np.dot(np.linalg.inv(np.dot(U_i, U_i.T) + lambda_V * np.identity(d_dims)),
                         np.dot(R[R[:, j] > 0, j], U_i.T))

    # Update the parameters dictionary with the updated U and V
    parameters['U_result'] = U
    parameters['V_result'] = V


Now let's implement the __Log-a posteriori__:

\begin{equation}
\large
L=-\frac 1 2 \left(\sum_{i=1}^N\sum_{j=1}^M(R_{ij}-U_i^TV_j)_{(i,j) \in \Omega_{R_{ij}}}^2+\lambda_U\sum_{i=1}^N\|U_i\|_{Fro}^2+\lambda_V\sum_{j=1}^M\|V_j\|_{Fro}^2\right)
\end{equation}

In [196]:
def log_a_posteriori():
    # Extract regularization parameters and result matrices from the 'parameters' dictionary
    lambda_U = parameters['lambda_U']
    lambda_V = parameters['lambda_V']
    U_result = parameters['U_result']
    V_result = parameters['V_result']
    R = parameters['R']

    UV = np.dot(U_result.T, V_result)

    # Compute the difference between observed ratings R and UV where R is greater than 0
    R_UV = (R[R > 0] - UV[R > 0])

    # Compute and return the negative log of the a posteriori probability
    return -0.5 * (np.sum(np.dot(R_UV, R_UV.T)) + lambda_U * np.sum(np.dot(U_result, U_result.T)) + \
                   lambda_V * np.sum(np.dot(V_result, V_result.T)))


For the purposes of __scaling__, we need the maximum and minimum rating values.

In [197]:
def update_max_min_ratings():
    """
    Update the maximum and minimum rating values in the parameters dictionary.
    """
    # Retrieve the U_result and V_result parameters
    U = parameters['U_result']
    V = parameters['V_result']

    # Compute the dot product of U_result and V_result
    R = U.T @ V

    # Find the minimum and maximum values in R
    min_rating = np.min(R)
    max_rating = np.max(R)

    # Store the minimum and maximum ratings in the parameters dictionary
    parameters['min_rating'] = min_rating
    parameters['max_rating'] = max_rating


The __predict__ function allows us to predict the rating value given the __user_id__ and the __movie_id__ parameters. The value has been scaled within the range 0-5

In [198]:
def predict(user_id, movie_id):
    # Extract optimized matrices U and V from 'parameters' dictionary
    U = parameters['U_result']
    V = parameters['V_result']

    user_id = int(user_id)
    movie_id = int(movie_id)

    # Compute the predicted rating for the given user and movie 
    r_ij = U[:, user_id-1].T.reshape(1, -1) @ V[:, movie_id-1].reshape(-1, 1) 

    max_rating = parameters['max_rating']
    min_rating = parameters['min_rating']

    # Normalize the predicted rating to be between 0 and 5 if possible, else return 0
    return 0 if max_rating == min_rating else ((r_ij[0][0] - min_rating) / (max_rating - min_rating)) * 5.0


In [199]:
def compare_results(n_users, n_movies):
    R = parameters['R']
    U_result = parameters['U_result']
    V_result = parameters['V_result']
    
    # Compute the predicted ratings matrix by taking the dot product of U_result and V_result
    R_result = U_result.T @ V_result
    
    # Compute the Frobenius norm of the difference between actual and predicted ratings matrices, 
    # and normalize it by the square root of the total number of ratings
    diff_norm_R = np.linalg.norm(R - R_result, 'fro')/np.sqrt(n_movies*n_users) 
    
    return diff_norm_R


The __evaluate__ function will calculate the __RMSE__ of the model given a dataset (train or test).

In [200]:
def evaluate(dataset):
    ground_truths = []
    predictions = []

    # Iterate over each row in the dataset
    for index, row in dataset.iterrows():    
        # Append the actual rating to the ground_truths list
        ground_truths.append(row.loc['rating'])
        # Predict the rating using our model and append it to the predictions list
        predictions.append(predict(row.loc['userID'], row.loc['movieID'])) 

    # Compute and return the root mean squared error (RMSE) between the ground truth and predicted ratings
    return mean_squared_error(ground_truths, predictions, squared=False)


The __train__ function implements the code necessary for training the model as well as recording the __RMSE__ values on the training and testing sets.

In [201]:
def train(train_set, test_set, n_users, n_movies, d_dims):
    log_aps = []
    rmse_train = []
    rmse_test = []

    update_max_min_ratings()
    
    # Compute and append RMSE for training and test set before training begins
    rmse_train.append(evaluate(train_set))
    rmse_test.append(evaluate(test_set))
    
    k = 0
    
    # The training process will continue until the absolute difference between consecutive 
    # log a posteriori values is less than 0.0001
    while True:

        if len(log_aps) > 2:
            if np.abs(log_aps[-1] - log_aps[-2]) < 0.0001: 
                break
        update_parameters(n_users, n_movies, d_dims)
        log_ap = log_a_posteriori()
        log_aps.append(log_ap)
        
        k += 1
        if (k + 1) % 25 == 0: 
            update_max_min_ratings()
            rmse_train.append(evaluate(train_set))
            rmse_test.append(evaluate(test_set))
            print('Log p a-posteriori at iteration', k + 1, ':', log_ap)

    update_max_min_ratings()
    
    # Compute the normalized difference between the actual and predicted ratings
    diff_norm_R = compare_results(n_users, n_movies)
    
    return log_aps, rmse_train, rmse_test, diff_norm_R 


Let's train our model!

In [216]:
def real_data_dimension_experiment(d_vals, n_users, n_items, ratings_matrix):    
    data = []
    np.set_printoptions(precision = 6)

    for i in range(len(d_vals)):
        
        # Initialize parameters using the current dimension and the ratings matrix
        init_params_real_data(sigma_u/sigma, sigma_v/sigma, mu, sigma, d_vals[i], n_users, n_items, ratings_matrix) 
        
        df_ratings = create_real_data_df(n_users, n_items) 
        train_set, test_set = split_train_test(df_ratings, train_size=0.95)
        _, rmse_train, rmse_test, diff_R = train(train_set, test_set, n_users, n_items, d_vals[i]) 
        
        row= [n_users, n_items, d_vals[i], diff_R, rmse_train[-1], rmse_test[-1]]
        data.append(row)       

    return data


In [214]:
def plot_results(data, group_nbr):
    df = pd.DataFrame(data, columns = ['N', 'M', 'D', 'diff_R', 'rmse_train', 'rmse_test']) 
    df.drop(['N', 'M'], axis=1, inplace=True)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Plot 'D' against 'diff_R'
    ax1.plot(df['D'], df['diff_R'], marker='o')
    ax1.set_xlabel('D')
    ax1.set_ylabel('diff_R')

    # Set the title based on group number
    if group_nbr is np.nan:
        ax1.set_title('Variation of diff_R for varying D')
    else:
        ax1.set_title(f'Variation of diff_R for varying D for group {group_nbr}')

    # Plot 'D' against 'rmse_train' and 'rmse_test'
    ax2.plot(df['D'], df['rmse_train'], marker='o', label='rmse_train')
    ax2.plot(df['D'], df['rmse_test'], marker='o', label='rmse_test')
    ax2.set_xlabel('D')
    ax2.set_ylabel('rmse')

    # Set the title based on group number
    if group_nbr is np.nan:
        ax2.set_title('Variation of rmse for varying D')
    else:
        ax2.set_title(f'Variation of rmse for varying D for group {group_nbr}')
    ax2.legend()

    plt.show() 


In [ ]:
d_vals = [5, 25, 50, 100, 150, 200] 
ratings_matrix, n_users, n_items = preprocess_data(ratings_df)
data = real_data_dimension_experiment(d_vals, n_users, n_items, ratings_matrix) 
plot_results(data, np.nan)

In [ ]:
group_dfs = [group_0_df, group_1_df, group_2_df, group_3_df, group_4_df]
group_nbrs = [0, 1, 2, 3, 4]
d_vals = [5, 10, 25, 30,40, 50, 60] 

for group_df, group_nbr in zip(group_dfs, group_nbrs):
    ratings_matrix, n_users, n_items = preprocess_data(group_df)
    data = real_data_dimension_experiment(d_vals, n_users, n_items, ratings_matrix)
    plot_results(data, group_nbr)
